In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
import os

pd.options.display.max_columns = None

In [4]:
# Путь к директории с данными

data_dir = '/home/codecraftman/notebook3.8/bin/track_first/track_1/'
os.listdir(data_dir)

['test.csv',
 'hydro_coord.csv',
 'meteo_1month.csv',
 'additional',
 'meteo_coord.csv',
 'ice_saw.csv',
 'train.csv',
 'test_task_1_random.csv',
 'meteo_3hours.csv',
 'meteo_1day.csv',
 'hydro_1day.csv',
 'reference_water_codes.csv',
 'reference_horiz_visib.csv']

In [5]:
# Загрузим всё
main_df = pd.read_csv(data_dir + 'train.csv')

# Метео
mc = pd.read_csv(data_dir + 'meteo_coord.csv')
mld = pd.read_csv(data_dir + 'meteo_1day.csv')
mlm = pd.read_csv(data_dir + 'meteo_1month.csv')
rhv = pd.read_csv(data_dir + 'reference_horiz_visib.csv')
m3h = pd.read_csv(data_dir + 'meteo_3hours.csv') # добавил метео 3 часа

# Гидро
hc = pd.read_csv(data_dir + 'hydro_coord.csv')
hld = pd.read_csv(data_dir + 'hydro_1day.csv',
                   parse_dates=['date'])
rwc = pd.read_csv(data_dir + 'reference_water_codes.csv')

In [6]:
# Подтянем ближайшую к гидростанции метеостанцию

from geopy.distance import geodesic
import re

def merge_coord(df):
    df['lat_long'] = df[['lat', 'lon']].apply(tuple, axis=1)
    return df

def stat_km(point, stat_list):
    stations_list=stat_list
    lst=[]
    if pd.isnull(point):
        lst.append(np.nan)
    else:
        for i in stations_list['lat_long']:
            x=geodesic(point, i).km
            lst.append(x)
            stations_list['dist']=pd.DataFrame(lst)
        y=stations_list['station_id'][stations_list['dist'] == stations_list['dist'].min()]
        y=y.to_string()
        y=re.sub("^[0-9]+", "", y)
        y=re.sub(" +", "", y)
        return int(y)

hc = merge_coord(hc)
mc = merge_coord(mc)
hc['closest_hydro'] = mc.lat_long.apply(lambda x: stat_km(x, mc))

In [7]:
final_table_columns = ['station_id','year_local', 'month_local', 'day_local','air_temperature' ]
m3h = m3h[m3h.columns.intersection(final_table_columns)]
#mld = meteo_df_3hours_before_final
#mld.rename(columns={'year_local':'year', 'month_local':'month', 'day_local':'day', 'date_local':'date'}, inplace=True)

In [8]:
mld = m3h.merge(hc[['station_id', 'closest_hydro']], left_on=['station_id'],
               right_on='closest_hydro', how='right')


In [9]:
mld.drop(['station_id_x', 'closest_hydro'], axis=1, inplace=True)

In [10]:
mld.rename({'station_id_y':'station_id','year_local':'year', 'month_local':'month', 'day_local':'day', 'date_local':'date'},axis=1, inplace=True)

In [11]:
mld.head()

,year,month,day,air_temperature,station_id
0,1985,1,1,-42.5,3019
1,1985,1,1,-43.0,3019
2,1985,1,1,-42.1,3019
3,1985,1,1,-43.1,3019
4,1985,1,1,-45.1,3019


### Разделим годы на train и test сбалансированно по суммарной продолжительности заторных событий

In [12]:
# Суммарное количество заторов в году
jams_by_year = main_df.groupby('year').sum()['ice_jam'].to_frame().reset_index()
jams_by_year['ice_jam'].describe()

count    26.000000
mean      5.307692
std       3.792300
min       0.000000
25%       3.000000
50%       5.000000
75%       8.000000
max      16.000000
Name: ice_jam, dtype: float64

In [13]:
# Разделим на бины по квартилям
bins = [-1, 3, 5, 8, 16]
jams_by_year['ice_jam_bins'] = pd.cut(jams_by_year['ice_jam'], bins)
X_length = jams_by_year[['year', 'ice_jam']]
y_length = jams_by_year['ice_jam_bins']
y_length.value_counts()

(-1, 3]    9
(3, 5]     7
(5, 8]     6
(8, 16]    4
Name: ice_jam_bins, dtype: int64

In [15]:
# Разделим годы на трейн и тест
X_train, X_test, y_train, y_test = train_test_split(X_length, 
                                                    y_length, 
                                                    test_size=0.3,  
                                                    stratify=y_length, 
                                                    random_state=42)

### Соберем фичи из гидроданных

<div class="alert alert-block alert-warning">
Мы не можем использовать данные из будущего: всё, что происходит после заторного периода, относится уже к следующему году.
</div>

In [16]:
# Внесем не темпоральные данные
main_df = pd.merge(main_df, hc[['station_id', 
                                'distance_from_source', 
                                'drainage_area', 
                                'z_null']], on='station_id', how='left')

# Возьмем также данные из ежедневных наблюдений
hld = pd.read_csv(data_dir + 'hydro_1day.csv',
                   parse_dates=['date'])
hld.head(1)

,year,station_id,month,day,date,stage_avg,stage_min,stage_max,temp,water_code,ice_thickness,snow_height,place,discharge
0,2000,3019,1,1,2000-01-01,74.0,74.0,74.0,NaN,46,NaN,NaN,NaN,NaN


In [17]:
hld['station_id'] = hld.station_id.astype(int)
mld['station_id'] = mld['station_id'].astype(int)

In [18]:
hld.head()

,year,station_id,month,day,date,stage_avg,stage_min,stage_max,temp,water_code,ice_thickness,snow_height,place,discharge
0,2000,3019,1,1,2000-01-01,74.0,74.0,74.0,NaN,46,NaN,NaN,NaN,NaN
1,2000,3019,1,2,2000-01-02,70.0,70.0,70.0,NaN,46,NaN,NaN,NaN,NaN
2,2000,3019,1,3,2000-01-03,67.0,67.0,67.0,NaN,46,NaN,NaN,NaN,NaN
3,2000,3019,1,4,2000-01-04,64.0,64.0,64.0,NaN,46,NaN,NaN,NaN,NaN
4,2000,3019,1,5,2000-01-05,60.0,60.0,60.0,NaN,46,NaN,NaN,NaN,NaN


In [19]:
mld.head()

,year,month,day,air_temperature,station_id
0,1985,1,1,-42.5,3019
1,1985,1,1,-43.0,3019
2,1985,1,1,-42.1,3019
3,1985,1,1,-43.1,3019
4,1985,1,1,-45.1,3019


In [20]:
mld.rename(columns={'year_local':'year', 'month_local':'month', 'day_local':'day'}, inplace=True)
hld = hld.merge(mld, on=['station_id', 'year', 'month', 'day'], how='inner')
hld = hld.fillna(0)
hld.tail()

,year,station_id,month,day,date,stage_avg,stage_min,stage_max,temp,water_code,ice_thickness,snow_height,place,discharge,air_temperature
1758499,1991,3106,7,212,1991-07-31,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,2670.0,10.8
1758500,1991,3106,7,212,1991-07-31,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,2670.0,13.6
1758501,1991,3106,7,212,1991-07-31,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,2670.0,13.9
1758502,1991,3106,7,212,1991-07-31,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,2670.0,15.1
1758503,1991,3106,7,212,1991-07-31,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,2670.0,15.2


### Скорректируем год, в который доступно наблюдение

In [21]:
# Определим для наблюдения год, в который оно доступно

# Маска-окно между заторным периодом и концом года
def after_jam_window(row, local=False):
    if local:
        month = row.month_local
        day = row.date_local.day
    else:
        month = row.month
        day = row.date.day
    return (((month == 6) and (day > 3))
            or (month in [7, 8, 9, 10, 11, 12]))

# Год относительно бизнес-логики
def target_year(row, local=False):
    if local:
        year = row.year_local
    else:
        year = row.year
    if after_jam_window(row):
        return year + 1
    else:
        return year
    
hld['target_year'] = hld.apply(target_year, axis=1)

# Календарный год и день больше не нужны
hld.drop(columns=['year', 'date', 'day'], axis=1, inplace=True)

In [22]:
# Гидро – сделаем ресэмплинг по месяцу

index = ['station_id', 'month', 'target_year']

hld_mean = hld.groupby(index).mean().add_prefix('mean_').reset_index()
hld_max = hld.groupby(index).max().add_prefix('max_').reset_index()
hld_min = hld.groupby(index).min().add_prefix('min_').reset_index()
hld_std = hld.groupby(index).std().add_prefix('std_').reset_index()
data_frames = [hld_mean, hld_max, hld_min, hld_std]

hydro_monthly = pd.concat(data_frames, axis=1)
hydro_monthly = hydro_monthly.loc[:,~hydro_monthly.columns.duplicated()]
hydro_monthly.sort_values(index).head()

,station_id,month,target_year,mean_stage_avg,mean_stage_min,mean_stage_max,mean_temp,mean_ice_thickness,mean_snow_height,mean_place,mean_discharge,mean_air_temperature,max_stage_avg,max_stage_min,max_stage_max,max_temp,max_ice_thickness,max_snow_height,max_place,max_discharge,max_air_temperature,min_stage_avg,min_stage_min,min_stage_max,min_temp,min_ice_thickness,min_snow_height,min_place,min_discharge,min_air_temperature,std_stage_avg,std_stage_min,std_stage_max,std_temp,std_ice_thickness,std_snow_height,std_place,std_discharge,std_air_temperature
0,3019,1,1985,-25.709677,-25.709677,-25.580645,0.0,10.612903,6.709677,0.193548,0.0,-45.302823,-23.0,-23.0,-23.0,0.0,58.0,44.0,1.0,0.0,-28.5,-28.0,-28.0,-28.0,0.0,0.0,0.0,0.0,0.0,-53.8,1.821039,1.821039,1.795617,0.0,21.739238,13.891875,0.395878,0.0,5.583974
1,3019,1,1986,56.322581,56.000000,56.451613,0.0,8.516129,5.774194,0.193548,0.0,-40.791935,74.0,74.0,74.0,0.0,47.0,34.0,1.0,0.0,-22.2,37.0,37.0,37.0,0.0,0.0,0.0,0.0,0.0,-53.6,12.721590,12.582360,12.727830,0.0,17.453923,11.898926,0.395878,0.0,7.682412
2,3019,1,1987,9.032258,8.516129,9.161290,0.0,9.322581,3.580645,0.193548,0.0,-42.455242,38.0,37.0,38.0,0.0,51.0,23.0,1.0,0.0,-20.1,-5.0,-5.0,-5.0,0.0,0.0,0.0,0.0,0.0,-56.1,13.543115,13.187826,13.663621,0.0,19.094289,7.441101,0.395878,0.0,9.796900
3,3019,1,1988,28.806452,28.516129,28.806452,0.0,9.096774,5.290323,0.193548,0.0,-36.610887,39.0,39.0,39.0,0.0,58.0,36.0,1.0,0.0,-10.5,19.0,19.0,19.0,0.0,0.0,0.0,0.0,0.0,-55.4,6.030526,5.908183,6.030526,0.0,18.930694,11.144087,0.395878,0.0,11.538519
4,3019,1,1989,82.741935,82.225806,82.774194,0.0,9.225806,7.774194,0.193548,0.0,-38.075806,97.0,96.0,97.0,0.0,55.0,48.0,1.0,0.0,-17.0,67.0,66.0,67.0,0.0,0.0,0.0,0.0,0.0,-49.8,9.794546,9.885858,9.790385,0.0,18.992792,16.060028,0.395878,0.0,8.336237


In [23]:
def make_features(df):
    station, target_year = df.name
    result = pd.DataFrame()
    for month, mdf in df.groupby('month'):
        m_feats = mdf[df.columns[4:]].add_prefix(str(month) + '_').reset_index(drop=True)
        result = pd.concat([result, m_feats], axis=1)
    return result.reset_index(drop=True)
        
hydro_features = hydro_monthly.groupby(['station_id', 'target_year']).apply(make_features)
hydro_features = hydro_features.reset_index(level=2, drop=True).reset_index()
hydro_features.dropna(how='all', axis=1, inplace=True)
hydro_features

,station_id,target_year,1_mean_stage_min,1_mean_stage_max,1_mean_temp,1_mean_ice_thickness,1_mean_snow_height,1_mean_place,1_mean_discharge,1_mean_air_temperature,1_max_stage_avg,1_max_stage_min,1_max_stage_max,1_max_temp,1_max_ice_thickness,1_max_snow_height,1_max_place,1_max_discharge,1_max_air_temperature,1_min_stage_avg,1_min_stage_min,1_min_stage_max,1_min_temp,1_min_ice_thickness,1_min_snow_height,1_min_place,1_min_discharge,1_min_air_temperature,1_std_stage_avg,1_std_stage_min,1_std_stage_max,1_std_temp,1_std_ice_thickness,1_std_snow_height,1_std_place,1_std_discharge,1_std_air_temperature,2_mean_stage_min,2_mean_stage_max,2_mean_temp,2_mean_ice_thickness,2_mean_snow_height,2_mean_place,2_mean_discharge,2_mean_air_temperature,2_max_stage_avg,2_max_stage_min,2_max_stage_max,2_max_temp,2_max_ice_thickness,2_max_snow_height,2_max_place,2_max_discharge,2_max_air_temperature,2_min_stage_avg,2_min_stage_min,2_min_stage_max,2_min_temp,2_min_ice_thickness,2_min_snow_height,2_min_place,2_min_discharge,2_min_air_temperature,2_std_stage_avg,2_std_stage_min,2_std_stage_max,2_std_temp,2_std_ice_thickness,2_std_snow_height,2_std_place,2_std_discharge,2_std_air_temperature,6_mean_stage_min,6_mean_stage_max,6_mean_temp,6_mean_ice_thickness,6_mean_snow_height,6_mean_place,6_mean_discharge,6_mean_air_temperature,6_max_stage_avg,6_max_stage_min,6_max_stage_max,6_max_temp,6_max_ice_thickness,6_max_snow_height,6_max_place,6_max_discharge,6_max_air_temperature,6_min_stage_avg,6_min_stage_min,6_min_stage_max,6_min_temp,6_min_ice_thickness,6_min_snow_height,6_min_place,6_min_discharge,6_min_air_temperature,6_std_stage_avg,6_std_stage_min,6_std_stage_max,6_std_temp,6_std_ice_thickness,6_std_snow_height,6_std_place,6_std_discharge,6_std_air_temperature,7_mean_stage_min,7_mean_stage_max,7_mean_temp,7_mean_ice_thickness,7_mean_snow_height,7_mean_place,7_mean_discharge,7_mean_air_temperature,7_max_stage_avg,7_max_stage_min,7_max_stage_max,7_max_temp,7_max_ice_thickness,7_max_snow_height,7_max_place,7_max_discharge,7_max_air_temperature,7_min_stage_avg,7_min_stage_min,7_min_stage_max,7_min_temp,7_min_ice_thickness,7_min_snow_height,7_min_place,7_min_discharge,7_min_air_temperature,7_std_stage_avg,7_std_stage_min,7_std_stage_max,7_std_temp,7_std_ice_thickness,7_std_snow_height,7_std_place,7_std_discharge,7_std_air_temperature,8_mean_stage_min,8_mean_stage_max,8_mean_temp,8_mean_ice_thickness,8_mean_snow_height,8_mean_place,8_mean_discharge,8_mean_air_temperature,8_max_stage_avg,8_max_stage_min,8_max_stage_max,8_max_temp,8_max_ice_thickness,8_max_snow_height,8_max_place,8_max_discharge,8_max_air_temperature,8_min_stage_avg,8_min_stage_min,8_min_stage_max,8_min_temp,8_min_ice_thickness,8_min_snow_height,8_min_place,8_min_discharge,8_min_air_temperature,8_std_stage_avg,8_std_stage_min,8_std_stage_max,8_std_temp,8_std_ice_thickness,8_std_snow_height,8_std_place,8_std_discharge,8_std_air_temperature,9_mean_stage_min,9_mean_stage_max,9_mean_temp,9_mean_ice_thickness,9_mean_snow_height,9_mean_place,9_mean_discharge,9_mean_air_temperature,9_max_stage_avg,9_max_stage_min,9_max_stage_max,9_max_temp,9_max_ice_thickness,9_max_snow_height,9_max_place,9_max_discharge,9_max_air_temperature,9_min_stage_avg,9_min_stage_min,9_min_stage_max,9_min_temp,9_min_ice_thickness,9_min_snow_height,9_min_place,9_min_discharge,9_min_air_temperature,9_std_stage_avg,9_std_stage_min,9_std_stage_max,9_std_temp,9_std_ice_thickness,9_std_snow_height,9_std_place,9_std_discharge,9_std_air_temperature,10_mean_stage_min,10_mean_stage_max,10_mean_temp,10_mean_ice_thickness,10_mean_snow_height,10_mean_place,10_mean_discharge,10_mean_air_temperature,10_max_stage_avg,10_max_stage_min,10_max_stage_max,10_max_temp,10_max_ice_thickness,10_max_snow_height,10_max_place,10_max_discharge,10_max_air_temperature,10_min_stage_avg,10_min_stage_min,10_min_stage_max,10_min_temp,10_min_ice_thickness,10_min_snow_height,10_min_place,10_min_discharge,10_min_air_temperature,10_std_stage_avg,10_std_stage_min,10_

In [147]:
list(hydro_features.columns)

['station_id',
 'target_year',
 '1_mean_stage_min',
 '1_mean_stage_max',
 '1_mean_temp',
 '1_mean_ice_thickness',
 '1_mean_snow_height',
 '1_mean_place',
 '1_mean_discharge',
 '1_mean_air_temperature',
 '1_max_stage_avg',
 '1_max_stage_min',
 '1_max_stage_max',
 '1_max_temp',
 '1_max_ice_thickness',
 '1_max_snow_height',
 '1_max_place',
 '1_max_discharge',
 '1_max_air_temperature',
 '1_min_stage_avg',
 '1_min_stage_min',
 '1_min_stage_max',
 '1_min_temp',
 '1_min_ice_thickness',
 '1_min_snow_height',
 '1_min_place',
 '1_min_discharge',
 '1_min_air_temperature',
 '1_std_stage_avg',
 '1_std_stage_min',
 '1_std_stage_max',
 '1_std_temp',
 '1_std_ice_thickness',
 '1_std_snow_height',
 '1_std_place',
 '1_std_discharge',
 '1_std_air_temperature',
 '2_mean_stage_min',
 '2_mean_stage_max',
 '2_mean_temp',
 '2_mean_ice_thickness',
 '2_mean_snow_height',
 '2_mean_place',
 '2_mean_discharge',
 '2_mean_air_temperature',
 '2_max_stage_avg',
 '2_max_stage_min',
 '2_max_stage_max',
 '2_max_temp',
 '2

### Соберем фичи в основной датасет

<div class="alert alert-block alert-success">
Важно: merge делаем по target_year.
</div>

In [24]:
main_df = pd.merge(main_df, hydro_features, left_on=['year', 'station_id'],
                   right_on=['target_year', 'station_id'],
                   how='left')
cols = main_df.columns.to_list()
main_df = main_df[cols[:3] + [cols[7]] + cols[5:7] + cols[8:] + [cols[3]]]
#main_df.dropna(how='any',inplace=True)
main_df.head(2)

,year,station_id,day,target_year,drainage_area,z_null,1_mean_stage_min,1_mean_stage_max,1_mean_temp,1_mean_ice_thickness,1_mean_snow_height,1_mean_place,1_mean_discharge,1_mean_air_temperature,1_max_stage_avg,1_max_stage_min,1_max_stage_max,1_max_temp,1_max_ice_thickness,1_max_snow_height,1_max_place,1_max_discharge,1_max_air_temperature,1_min_stage_avg,1_min_stage_min,1_min_stage_max,1_min_temp,1_min_ice_thickness,1_min_snow_height,1_min_place,1_min_discharge,1_min_air_temperature,1_std_stage_avg,1_std_stage_min,1_std_stage_max,1_std_temp,1_std_ice_thickness,1_std_snow_height,1_std_place,1_std_discharge,1_std_air_temperature,2_mean_stage_min,2_mean_stage_max,2_mean_temp,2_mean_ice_thickness,2_mean_snow_height,2_mean_place,2_mean_discharge,2_mean_air_temperature,2_max_stage_avg,2_max_stage_min,2_max_stage_max,2_max_temp,2_max_ice_thickness,2_max_snow_height,2_max_place,2_max_discharge,2_max_air_temperature,2_min_stage_avg,2_min_stage_min,2_min_stage_max,2_min_temp,2_min_ice_thickness,2_min_snow_height,2_min_place,2_min_discharge,2_min_air_temperature,2_std_stage_avg,2_std_stage_min,2_std_stage_max,2_std_temp,2_std_ice_thickness,2_std_snow_height,2_std_place,2_std_discharge,2_std_air_temperature,6_mean_stage_min,6_mean_stage_max,6_mean_temp,6_mean_ice_thickness,6_mean_snow_height,6_mean_place,6_mean_discharge,6_mean_air_temperature,6_max_stage_avg,6_max_stage_min,6_max_stage_max,6_max_temp,6_max_ice_thickness,6_max_snow_height,6_max_place,6_max_discharge,6_max_air_temperature,6_min_stage_avg,6_min_stage_min,6_min_stage_max,6_min_temp,6_min_ice_thickness,6_min_snow_height,6_min_place,6_min_discharge,6_min_air_temperature,6_std_stage_avg,6_std_stage_min,6_std_stage_max,6_std_temp,6_std_ice_thickness,6_std_snow_height,6_std_place,6_std_discharge,6_std_air_temperature,7_mean_stage_min,7_mean_stage_max,7_mean_temp,7_mean_ice_thickness,7_mean_snow_height,7_mean_place,7_mean_discharge,7_mean_air_temperature,7_max_stage_avg,7_max_stage_min,7_max_stage_max,7_max_temp,7_max_ice_thickness,7_max_snow_height,7_max_place,7_max_discharge,7_max_air_temperature,7_min_stage_avg,7_min_stage_min,7_min_stage_max,7_min_temp,7_min_ice_thickness,7_min_snow_height,7_min_place,7_min_discharge,7_min_air_temperature,7_std_stage_avg,7_std_stage_min,7_std_stage_max,7_std_temp,7_std_ice_thickness,7_std_snow_height,7_std_place,7_std_discharge,7_std_air_temperature,8_mean_stage_min,8_mean_stage_max,8_mean_temp,8_mean_ice_thickness,8_mean_snow_height,8_mean_place,8_mean_discharge,8_mean_air_temperature,8_max_stage_avg,8_max_stage_min,8_max_stage_max,8_max_temp,8_max_ice_thickness,8_max_snow_height,8_max_place,8_max_discharge,8_max_air_temperature,8_min_stage_avg,8_min_stage_min,8_min_stage_max,8_min_temp,8_min_ice_thickness,8_min_snow_height,8_min_place,8_min_discharge,8_min_air_temperature,8_std_stage_avg,8_std_stage_min,8_std_stage_max,8_std_temp,8_std_ice_thickness,8_std_snow_height,8_std_place,8_std_discharge,8_std_air_temperature,9_mean_stage_min,9_mean_stage_max,9_mean_temp,9_mean_ice_thickness,9_mean_snow_height,9_mean_place,9_mean_discharge,9_mean_air_temperature,9_max_stage_avg,9_max_stage_min,9_max_stage_max,9_max_temp,9_max_ice_thickness,9_max_snow_height,9_max_place,9_max_discharge,9_max_air_temperature,9_min_stage_avg,9_min_stage_min,9_min_stage_max,9_min_temp,9_min_ice_thickness,9_min_snow_height,9_min_place,9_min_discharge,9_min_air_temperature,9_std_stage_avg,9_std_stage_min,9_std_stage_max,9_std_temp,9_std_ice_thickness,9_std_snow_height,9_std_place,9_std_discharge,9_std_air_temperature,10_mean_stage_min,10_mean_stage_max,10_mean_temp,10_mean_ice_thickness,10_mean_snow_height,10_mean_place,10_mean_discharge,10_mean_air_temperature,10_max_stage_avg,10_max_stage_min,10_max_stage_max,10_max_temp,10_max_ice_thickness,10_max_snow_height,10_max_place,10_max_discharge,10_max_air_temperature,10_min_stage_avg,10_min_stage_min,10_min_stage_max,10_min_temp,10_min_ice_thickness,10_min_snow_height,10_min_place,10_min_discharge,10_min_air_temperature,10_std_

### Нормируем фичи

In [25]:
from sklearn.preprocessing import StandardScaler

ids, data, target = main_df[main_df.columns[:4]], main_df[main_df.columns[4:-1]], main_df[main_df.columns[-1]]

scaler = StandardScaler()
scaler.fit(data)
transformed_data = scaler.transform(data)
norm_df = pd.concat([ids, pd.DataFrame(transformed_data, columns = main_df.columns[4:-1]), target], axis=1)
norm_df

,year,station_id,day,target_year,drainage_area,z_null,1_mean_stage_min,1_mean_stage_max,1_mean_temp,1_mean_ice_thickness,1_mean_snow_height,1_mean_place,1_mean_discharge,1_mean_air_temperature,1_max_stage_avg,1_max_stage_min,1_max_stage_max,1_max_temp,1_max_ice_thickness,1_max_snow_height,1_max_place,1_max_discharge,1_max_air_temperature,1_min_stage_avg,1_min_stage_min,1_min_stage_max,1_min_temp,1_min_ice_thickness,1_min_snow_height,1_min_place,1_min_discharge,1_min_air_temperature,1_std_stage_avg,1_std_stage_min,1_std_stage_max,1_std_temp,1_std_ice_thickness,1_std_snow_height,1_std_place,1_std_discharge,1_std_air_temperature,2_mean_stage_min,2_mean_stage_max,2_mean_temp,2_mean_ice_thickness,2_mean_snow_height,2_mean_place,2_mean_discharge,2_mean_air_temperature,2_max_stage_avg,2_max_stage_min,2_max_stage_max,2_max_temp,2_max_ice_thickness,2_max_snow_height,2_max_place,2_max_discharge,2_max_air_temperature,2_min_stage_avg,2_min_stage_min,2_min_stage_max,2_min_temp,2_min_ice_thickness,2_min_snow_height,2_min_place,2_min_discharge,2_min_air_temperature,2_std_stage_avg,2_std_stage_min,2_std_stage_max,2_std_temp,2_std_ice_thickness,2_std_snow_height,2_std_place,2_std_discharge,2_std_air_temperature,6_mean_stage_min,6_mean_stage_max,6_mean_temp,6_mean_ice_thickness,6_mean_snow_height,6_mean_place,6_mean_discharge,6_mean_air_temperature,6_max_stage_avg,6_max_stage_min,6_max_stage_max,6_max_temp,6_max_ice_thickness,6_max_snow_height,6_max_place,6_max_discharge,6_max_air_temperature,6_min_stage_avg,6_min_stage_min,6_min_stage_max,6_min_temp,6_min_ice_thickness,6_min_snow_height,6_min_place,6_min_discharge,6_min_air_temperature,6_std_stage_avg,6_std_stage_min,6_std_stage_max,6_std_temp,6_std_ice_thickness,6_std_snow_height,6_std_place,6_std_discharge,6_std_air_temperature,7_mean_stage_min,7_mean_stage_max,7_mean_temp,7_mean_ice_thickness,7_mean_snow_height,7_mean_place,7_mean_discharge,7_mean_air_temperature,7_max_stage_avg,7_max_stage_min,7_max_stage_max,7_max_temp,7_max_ice_thickness,7_max_snow_height,7_max_place,7_max_discharge,7_max_air_temperature,7_min_stage_avg,7_min_stage_min,7_min_stage_max,7_min_temp,7_min_ice_thickness,7_min_snow_height,7_min_place,7_min_discharge,7_min_air_temperature,7_std_stage_avg,7_std_stage_min,7_std_stage_max,7_std_temp,7_std_ice_thickness,7_std_snow_height,7_std_place,7_std_discharge,7_std_air_temperature,8_mean_stage_min,8_mean_stage_max,8_mean_temp,8_mean_ice_thickness,8_mean_snow_height,8_mean_place,8_mean_discharge,8_mean_air_temperature,8_max_stage_avg,8_max_stage_min,8_max_stage_max,8_max_temp,8_max_ice_thickness,8_max_snow_height,8_max_place,8_max_discharge,8_max_air_temperature,8_min_stage_avg,8_min_stage_min,8_min_stage_max,8_min_temp,8_min_ice_thickness,8_min_snow_height,8_min_place,8_min_discharge,8_min_air_temperature,8_std_stage_avg,8_std_stage_min,8_std_stage_max,8_std_temp,8_std_ice_thickness,8_std_snow_height,8_std_place,8_std_discharge,8_std_air_temperature,9_mean_stage_min,9_mean_stage_max,9_mean_temp,9_mean_ice_thickness,9_mean_snow_height,9_mean_place,9_mean_discharge,9_mean_air_temperature,9_max_stage_avg,9_max_stage_min,9_max_stage_max,9_max_temp,9_max_ice_thickness,9_max_snow_height,9_max_place,9_max_discharge,9_max_air_temperature,9_min_stage_avg,9_min_stage_min,9_min_stage_max,9_min_temp,9_min_ice_thickness,9_min_snow_height,9_min_place,9_min_discharge,9_min_air_temperature,9_std_stage_avg,9_std_stage_min,9_std_stage_max,9_std_temp,9_std_ice_thickness,9_std_snow_height,9_std_place,9_std_discharge,9_std_air_temperature,10_mean_stage_min,10_mean_stage_max,10_mean_temp,10_mean_ice_thickness,10_mean_snow_height,10_mean_place,10_mean_discharge,10_mean_air_temperature,10_max_stage_avg,10_max_stage_min,10_max_stage_max,10_max_temp,10_max_ice_thickness,10_max_snow_height,10_max_place,10_max_discharge,10_max_air_temperature,10_min_stage_avg,10_min_stage_min,10_min_stage_max,10_min_temp,10_min_ice_thickness,10_min_snow_height,10_min_place,10_min_discharge,10_min_air_temperature,10_std_

In [26]:
# Разделим на трейн и тест исходя из target_year

test = norm_df[~norm_df.target_year.isin(X_train.year.to_list())].reset_index(drop=True).dropna()
train = norm_df[norm_df.target_year.isin(X_train.year.to_list())].reset_index(drop=True).dropna()

# target_year больше не нужна

test.drop(columns=['target_year'], inplace=True)
train.drop(columns=['target_year'], inplace=True)

In [27]:
train.shape[0]

6952

In [28]:
# Поделим данные на предикторы и таргет

X_train, y_train = train.iloc[:, :-1], train.ice_jam
X_test, y_test = test.iloc[:, :-1], test.ice_jam

In [173]:
items_to_drop = list(X_train.columns)

In [174]:
import re
pref = re.search("^[0-9]*_\w{3}_ice_\w*$")

for word in items_to_drop[:]:
    if re.search("^[0-9]*_\w{3}_ice_\w*$", word):
        items_to_drop.remove(word)

TypeError: search() missing 1 required positional argument: 'string'

In [67]:

for word in items_to_drop[:]:
    if re.search("^[0-9]*_\w*_snow_\w*$", word):
        items_to_drop.remove(word)


In [70]:

for word in items_to_drop[:]:
    if re.search("^[0-9]*_\w*_ice_\w*$", word):
        items_to_drop.remove(word)


In [73]:

for word in items_to_drop[:]:
    if re.search("^[0-9]*_\w*_stage_\w*$", word):
        items_to_drop.remove(word)


In [76]:

for word in items_to_drop[:]:
    if re.search("^[0-9]*_\w*_discharge$", word):
        items_to_drop.remove(word)


In [81]:

for word in items_to_drop[:]:
    if re.search("^[0-9]*_\w*_temp$", word):
        items_to_drop.remove(word)

In [247]:
X_train

,year,station_id,day,drainage_area,z_null,1_mean_stage_min,1_mean_stage_max,1_mean_temp,1_mean_ice_thickness,1_mean_snow_height,1_mean_place,1_mean_discharge,1_mean_air_temperature,1_max_stage_avg,1_max_stage_min,1_max_stage_max,1_max_temp,1_max_ice_thickness,1_max_snow_height,1_max_place,1_max_discharge,1_max_air_temperature,1_min_stage_avg,1_min_stage_min,1_min_stage_max,1_min_temp,1_min_ice_thickness,1_min_snow_height,1_min_place,1_min_discharge,1_min_air_temperature,1_std_stage_avg,1_std_stage_min,1_std_stage_max,1_std_temp,1_std_ice_thickness,1_std_snow_height,1_std_place,1_std_discharge,1_std_air_temperature,2_mean_stage_min,2_mean_stage_max,2_mean_temp,2_mean_ice_thickness,2_mean_snow_height,2_mean_place,2_mean_discharge,2_mean_air_temperature,2_max_stage_avg,2_max_stage_min,2_max_stage_max,2_max_temp,2_max_ice_thickness,2_max_snow_height,2_max_place,2_max_discharge,2_max_air_temperature,2_min_stage_avg,2_min_stage_min,2_min_stage_max,2_min_temp,2_min_ice_thickness,2_min_snow_height,2_min_place,2_min_discharge,2_min_air_temperature,2_std_stage_avg,2_std_stage_min,2_std_stage_max,2_std_temp,2_std_ice_thickness,2_std_snow_height,2_std_place,2_std_discharge,2_std_air_temperature,6_mean_stage_min,6_mean_stage_max,6_mean_temp,6_mean_ice_thickness,6_mean_snow_height,6_mean_place,6_mean_discharge,6_mean_air_temperature,6_max_stage_avg,6_max_stage_min,6_max_stage_max,6_max_temp,6_max_ice_thickness,6_max_snow_height,6_max_place,6_max_discharge,6_max_air_temperature,6_min_stage_avg,6_min_stage_min,6_min_stage_max,6_min_temp,6_min_ice_thickness,6_min_snow_height,6_min_place,6_min_discharge,6_min_air_temperature,6_std_stage_avg,6_std_stage_min,6_std_stage_max,6_std_temp,6_std_ice_thickness,6_std_snow_height,6_std_place,6_std_discharge,6_std_air_temperature,7_mean_stage_min,7_mean_stage_max,7_mean_temp,7_mean_ice_thickness,7_mean_snow_height,7_mean_place,7_mean_discharge,7_mean_air_temperature,7_max_stage_avg,7_max_stage_min,7_max_stage_max,7_max_temp,7_max_ice_thickness,7_max_snow_height,7_max_place,7_max_discharge,7_max_air_temperature,7_min_stage_avg,7_min_stage_min,7_min_stage_max,7_min_temp,7_min_ice_thickness,7_min_snow_height,7_min_place,7_min_discharge,7_min_air_temperature,7_std_stage_avg,7_std_stage_min,7_std_stage_max,7_std_temp,7_std_ice_thickness,7_std_snow_height,7_std_place,7_std_discharge,7_std_air_temperature,8_mean_stage_min,8_mean_stage_max,8_mean_temp,8_mean_ice_thickness,8_mean_snow_height,8_mean_place,8_mean_discharge,8_mean_air_temperature,8_max_stage_avg,8_max_stage_min,8_max_stage_max,8_max_temp,8_max_ice_thickness,8_max_snow_height,8_max_place,8_max_discharge,8_max_air_temperature,8_min_stage_avg,8_min_stage_min,8_min_stage_max,8_min_temp,8_min_ice_thickness,8_min_snow_height,8_min_place,8_min_discharge,8_min_air_temperature,8_std_stage_avg,8_std_stage_min,8_std_stage_max,8_std_temp,8_std_ice_thickness,8_std_snow_height,8_std_place,8_std_discharge,8_std_air_temperature,9_mean_stage_min,9_mean_stage_max,9_mean_temp,9_mean_ice_thickness,9_mean_snow_height,9_mean_place,9_mean_discharge,9_mean_air_temperature,9_max_stage_avg,9_max_stage_min,9_max_stage_max,9_max_temp,9_max_ice_thickness,9_max_snow_height,9_max_place,9_max_discharge,9_max_air_temperature,9_min_stage_avg,9_min_stage_min,9_min_stage_max,9_min_temp,9_min_ice_thickness,9_min_snow_height,9_min_place,9_min_discharge,9_min_air_temperature,9_std_stage_avg,9_std_stage_min,9_std_stage_max,9_std_temp,9_std_ice_thickness,9_std_snow_height,9_std_place,9_std_discharge,9_std_air_temperature,10_mean_stage_min,10_mean_stage_max,10_mean_temp,10_mean_ice_thickness,10_mean_snow_height,10_mean_place,10_mean_discharge,10_mean_air_temperature,10_max_stage_avg,10_max_stage_min,10_max_stage_max,10_max_temp,10_max_ice_thickness,10_max_snow_height,10_max_place,10_max_discharge,10_max_air_temperature,10_min_stage_avg,10_min_stage_min,10_min_stage_max,10_min_temp,10_min_ice_thickness,10_min_snow_height,10_min_place,10_min_discharge,10_min_air_temperature,10_std_stage_avg,10

In [29]:
model_clear = X_train

In [30]:
model_clear.drop(model_clear.columns[model_clear.columns.str.contains(r'^[0-9]*_\w{3}_ice_\w*$')], axis=1,inplace=True)


In [31]:
model_clear.drop(model_clear.columns[model_clear.columns.str.contains(r'^[0-9]*_\w*_snow_\w*$')], axis=1,inplace=True)


In [32]:
model_clear.drop(model_clear.columns[model_clear.columns.str.contains(r'^[0-9]*_\w*_ice_\w*$')], axis=1,inplace=True)


In [33]:
model_clear.drop(model_clear.columns[model_clear.columns.str.contains(r'^[0-9]*_\w*_stage_\w*$')], axis=1,inplace=True)


In [119]:
#model_clear.drop(model_clear.columns[model_clear.columns.str.contains(r'^[0-9]*_\w*_temp$')], axis=1,inplace=True)


In [34]:
model_clear.head()

,year,station_id,day,drainage_area,z_null,1_mean_temp,1_mean_place,1_mean_discharge,1_mean_air_temperature,1_max_temp,1_max_place,1_max_discharge,1_max_air_temperature,1_min_temp,1_min_place,1_min_discharge,1_min_air_temperature,1_std_temp,1_std_place,1_std_discharge,1_std_air_temperature,2_mean_temp,2_mean_place,2_mean_discharge,2_mean_air_temperature,2_max_temp,2_max_place,2_max_discharge,2_max_air_temperature,2_min_temp,2_min_place,2_min_discharge,2_min_air_temperature,2_std_temp,2_std_place,2_std_discharge,2_std_air_temperature,6_mean_temp,6_mean_place,6_mean_discharge,6_mean_air_temperature,6_max_temp,6_max_place,6_max_discharge,6_max_air_temperature,6_min_temp,6_min_place,6_min_discharge,6_min_air_temperature,6_std_temp,6_std_place,6_std_discharge,6_std_air_temperature,7_mean_temp,7_mean_place,7_mean_discharge,7_mean_air_temperature,7_max_temp,7_max_place,7_max_discharge,7_max_air_temperature,7_min_temp,7_min_place,7_min_discharge,7_min_air_temperature,7_std_temp,7_std_place,7_std_discharge,7_std_air_temperature,8_mean_temp,8_mean_place,8_mean_discharge,8_mean_air_temperature,8_max_temp,8_max_place,8_max_discharge,8_max_air_temperature,8_min_temp,8_min_place,8_min_discharge,8_min_air_temperature,8_std_temp,8_std_place,8_std_discharge,8_std_air_temperature,9_mean_temp,9_mean_place,9_mean_discharge,9_mean_air_temperature,9_max_temp,9_max_place,9_max_discharge,9_max_air_temperature,9_min_temp,9_min_place,9_min_discharge,9_min_air_temperature,9_std_temp,9_std_place,9_std_discharge,9_std_air_temperature,10_mean_temp,10_mean_place,10_mean_discharge,10_mean_air_temperature,10_max_temp,10_max_place,10_max_discharge,10_max_air_temperature,10_min_temp,10_min_place,10_min_discharge,10_min_air_temperature,10_std_temp,10_std_place,10_std_discharge,10_std_air_temperature,11_mean_temp,11_mean_place,11_mean_discharge,11_mean_air_temperature,11_max_temp,11_max_place,11_max_discharge,11_max_air_temperature,11_min_temp,11_min_place,11_min_discharge,11_min_air_temperature,11_std_temp,11_std_place,11_std_discharge,11_std_air_temperature,12_mean_temp,12_mean_place,12_mean_discharge,12_mean_air_temperature,12_max_temp,12_max_place,12_max_discharge,12_max_air_temperature,12_min_temp,12_min_place,12_min_discharge,12_min_air_temperature,12_std_temp,12_std_place,12_std_discharge,12_std_air_temperature
0,2000,3019,1,-1.277494,2.039361,0.0,-0.300853,-0.330884,-0.228598,0.0,-0.063989,-0.329791,0.459695,0.0,0.0,-0.33077,0.049724,0.0,-0.286342,-0.290306,-0.013706,0.0,-0.264775,-0.331855,0.310626,0.0,0.0,-0.331846,0.336318,0.0,0.0,-0.331614,0.709358,0.0,-0.264775,-0.272546,0.414686,0.660172,0.0,-0.322072,-0.861506,0.583158,0.0,-0.324814,-0.007487,0.952596,0.0,-0.322783,0.00611,-0.440665,0.0,-0.316882,-0.385316,1.169777,0.0,-0.322141,0.367262,1.451491,0.0,-0.317685,0.570478,0.236854,0.0,-0.322649,-0.197053,2.39014,0.0,-0.302046,0.98885,0.051218,0.0,-0.318364,-1.484799,0.373895,0.0,-0.317145,-1.267961,0.893391,0.0,-0.327532,-0.108419,-0.480547,0.0,-0.288731,0.207454,0.858333,0.0,-0.325431,-0.9383,0.4031,0.0,-0.321382,0.019471,0.501308,0.0,-0.327042,-2.036131,-0.385427,0.0,-0.297424,1.099854,-0.571368,-0.105498,-0.326588,-1.562546,-0.953321,-0.131791,-0.324283,-1.600582,-0.243573,0.0,-0.316077,-0.614828,-0.614485,-0.125517,-0.318807,0.281407,-0.091659,-0.053888,-0.332229,-0.413252,-0.18179,-0.009976,-0.330239,0.65774,-0.083025,0.0,-0.31628,-0.73875,-0.245526,0.008517,-0.324479,0.630211,0.0,-0.436874,-0.33093,-0.210977,0.0,-0.09259,-0.330396,0.576224,0.0,0.0,-0.331485,-0.77623,0.0,-0.383016,-0.286161,0.700256
1,2000,3019,2,-1.277494,2.039361,0.0,-0.300853,-0.330884,-0.228598,0.0,-0.063989,-0.329791,0.459695,0.0,0.0,-0.33077,0.049724,0.0,-0.286342,-0.290306,-0.013706,0.0,-0.264775,-0.331855,0.310626,0.0,0.0,-0.331846,0.336318,0.0,0.0,-0.331614,0.709358,0.0,-0.264775,-0.272546,0.414686,0.660172,0.0,-0.322072,-0.861506,0.583158,0.0,-0.324814,-0.007487,0.952596,0.0,-0.322783,0.00611,-0.440665,0.0,-0.316882,-0.385316,1.169777,0.0,-0.322141,0.367262,1.451491,0.0,

In [102]:
#model_clear.drop(pd.Series([x for x in items_to_drop]), axis=1, inplace=True)
#model_clear.drop(model_clear.columns[model_clear.columns.str.contains([str(x) for x in items_to_drop])], axis=1)

#model_clear = [df.drop([x for x in items_to_drop], axis=1) for df in model_clear]
#for df in model_clear:
    #df.drop(list(items_to_drop), axis=1, inplace=True)

In [35]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [66]:
model_clear.head()

,year,station_id,day,drainage_area,z_null,1_mean_temp,1_mean_place,1_mean_discharge,1_mean_air_temperature,1_max_temp,1_max_place,1_max_discharge,1_max_air_temperature,1_min_temp,1_min_place,1_min_discharge,1_min_air_temperature,1_std_temp,1_std_place,1_std_discharge,1_std_air_temperature,2_mean_temp,2_mean_place,2_mean_discharge,2_mean_air_temperature,2_max_temp,2_max_place,2_max_discharge,2_max_air_temperature,2_min_temp,2_min_place,2_min_discharge,2_min_air_temperature,2_std_temp,2_std_place,2_std_discharge,2_std_air_temperature,6_mean_temp,6_mean_place,6_mean_discharge,6_mean_air_temperature,6_max_temp,6_max_place,6_max_discharge,6_max_air_temperature,6_min_temp,6_min_place,6_min_discharge,6_min_air_temperature,6_std_temp,6_std_place,6_std_discharge,6_std_air_temperature,7_mean_temp,7_mean_place,7_mean_discharge,7_mean_air_temperature,7_max_temp,7_max_place,7_max_discharge,7_max_air_temperature,7_min_temp,7_min_place,7_min_discharge,7_min_air_temperature,7_std_temp,7_std_place,7_std_discharge,7_std_air_temperature,8_mean_temp,8_mean_place,8_mean_discharge,8_mean_air_temperature,8_max_temp,8_max_place,8_max_discharge,8_max_air_temperature,8_min_temp,8_min_place,8_min_discharge,8_min_air_temperature,8_std_temp,8_std_place,8_std_discharge,8_std_air_temperature,9_mean_temp,9_mean_place,9_mean_discharge,9_mean_air_temperature,9_max_temp,9_max_place,9_max_discharge,9_max_air_temperature,9_min_temp,9_min_place,9_min_discharge,9_min_air_temperature,9_std_temp,9_std_place,9_std_discharge,9_std_air_temperature,10_mean_temp,10_mean_place,10_mean_discharge,10_mean_air_temperature,10_max_temp,10_max_place,10_max_discharge,10_max_air_temperature,10_min_temp,10_min_place,10_min_discharge,10_min_air_temperature,10_std_temp,10_std_place,10_std_discharge,10_std_air_temperature,11_mean_temp,11_mean_place,11_mean_discharge,11_mean_air_temperature,11_max_temp,11_max_place,11_max_discharge,11_max_air_temperature,11_min_temp,11_min_place,11_min_discharge,11_min_air_temperature,11_std_temp,11_std_place,11_std_discharge,11_std_air_temperature,12_mean_temp,12_mean_place,12_mean_discharge,12_mean_air_temperature,12_max_temp,12_max_place,12_max_discharge,12_max_air_temperature,12_min_temp,12_min_place,12_min_discharge,12_min_air_temperature,12_std_temp,12_std_place,12_std_discharge,12_std_air_temperature
0,2000,3019,1,-1.277494,2.039361,0.0,-0.300853,-0.330884,-0.228598,0.0,-0.063989,-0.329791,0.459695,0.0,0.0,-0.33077,0.049724,0.0,-0.286342,-0.290306,-0.013706,0.0,-0.264775,-0.331855,0.310626,0.0,0.0,-0.331846,0.336318,0.0,0.0,-0.331614,0.709358,0.0,-0.264775,-0.272546,0.414686,0.660172,0.0,-0.322072,-0.861506,0.583158,0.0,-0.324814,-0.007487,0.952596,0.0,-0.322783,0.00611,-0.440665,0.0,-0.316882,-0.385316,1.169777,0.0,-0.322141,0.367262,1.451491,0.0,-0.317685,0.570478,0.236854,0.0,-0.322649,-0.197053,2.39014,0.0,-0.302046,0.98885,0.051218,0.0,-0.318364,-1.484799,0.373895,0.0,-0.317145,-1.267961,0.893391,0.0,-0.327532,-0.108419,-0.480547,0.0,-0.288731,0.207454,0.858333,0.0,-0.325431,-0.9383,0.4031,0.0,-0.321382,0.019471,0.501308,0.0,-0.327042,-2.036131,-0.385427,0.0,-0.297424,1.099854,-0.571368,-0.105498,-0.326588,-1.562546,-0.953321,-0.131791,-0.324283,-1.600582,-0.243573,0.0,-0.316077,-0.614828,-0.614485,-0.125517,-0.318807,0.281407,-0.091659,-0.053888,-0.332229,-0.413252,-0.18179,-0.009976,-0.330239,0.65774,-0.083025,0.0,-0.31628,-0.73875,-0.245526,0.008517,-0.324479,0.630211,0.0,-0.436874,-0.33093,-0.210977,0.0,-0.09259,-0.330396,0.576224,0.0,0.0,-0.331485,-0.77623,0.0,-0.383016,-0.286161,0.700256
1,2000,3019,2,-1.277494,2.039361,0.0,-0.300853,-0.330884,-0.228598,0.0,-0.063989,-0.329791,0.459695,0.0,0.0,-0.33077,0.049724,0.0,-0.286342,-0.290306,-0.013706,0.0,-0.264775,-0.331855,0.310626,0.0,0.0,-0.331846,0.336318,0.0,0.0,-0.331614,0.709358,0.0,-0.264775,-0.272546,0.414686,0.660172,0.0,-0.322072,-0.861506,0.583158,0.0,-0.324814,-0.007487,0.952596,0.0,-0.322783,0.00611,-0.440665,0.0,-0.316882,-0.385316,1.169777,0.0,-0.322141,0.367262,1.451491,0.0,

In [163]:
resharped_model_1 = model_clear.pivot_table(index='1_min_air_temperature', columns='1_max_air_temperature', values='drainage_area').round(1)

In [164]:
resharped_model_1 = resharped_model_1.fillna(0)

In [165]:
resharped_model_1.head(48)

1_max_air_temperature,-2.482895,-2.373459,-2.336981,-2.191067,-2.130270,-2.045154,-2.032994,-1.935719,-1.801965,-1.765486,-1.729008,-1.680370,-1.510138,-1.485819,-1.449340,-1.400702,-1.388543,-1.364224,-1.339905,-1.254789,-1.242629,-1.181832,-1.169673,-1.121035,-1.072397,-1.023759,-0.999440,-0.890005,-0.877846,-0.853527,-0.804889,-0.780570,-0.744091,-0.731932,-0.719773,-0.683294,-0.634656,-0.610337,-0.573859,-0.549540,-0.525221,-0.488743,-0.476583,-0.464424,-0.440105,-0.403627,-0.391467,-0.367148,-0.354989,-0.282032,-0.196916,-0.160437,-0.063162,-0.038843,-0.014524,-0.002364,0.009795,0.021955,0.070592,0.082752,0.107071,0.119230,0.131390,0.155709,0.204347,0.216506,0.228666,0.240825,0.265144,0.277303,0.289463,0.313782,0.325941,0.338101,0.350260,0.362420,0.374579,0.386739,0.423217,0.435376,0.447536,0.459695,0.471855,0.520493,0.544812,0.556971,0.569130,0.605609,0.617768,0.629928,0.642087,0.654247,0.702885,0.727203,0.739363,0.751522,0.788001,0.812320,0.824479,0.848798,0.909595,0.921755,0.946074,0.958233,0.970393,0.982552,0.994712,1.079828,1.116306,1.164944,1.201422,1.310858,1.335177,1.371655,1.444612,1.468931,1.493250,1.529728,1.578366,1.894512
1_min_air_temperature,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
-1.852235,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-1.765782,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-1.722556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-1.657716,0.0,0.0,-0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-1.614490,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-1.549651,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [166]:
from statsmodels.tsa.arima_model import ARIMA

In [167]:
model = ARIMA(resharped_model_1.values.reshape(-1).tolist(), order=(1, 0, 0))

/home/codecraftman/notebook3.8/lib/python3.8/site-packages/statsmodels/tsa/arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


In [168]:
model_fit = model.fit(disp=False)

In [169]:
predictions = model_fit.predict(len(X_test))

In [170]:
test_ = pd.DataFrame(X_test)

In [171]:
predictions

array([6.34919799e-05, 6.34919799e-05, 6.34919799e-05, ...,
       6.27463576e-05, 6.34919799e-05, 6.34919799e-05])

In [132]:
type(predictions)

numpy.ndarray

In [172]:
prediction_Series = pd.Series(predictions)

In [173]:
prediction_Series.unique()

array([6.34919799e-05, 6.34919799e-05, 6.33055743e-05, 6.35541151e-05,
       6.38026558e-05, 6.36473178e-05, 6.34919799e-05, 6.27463576e-05,
       6.34298447e-05, 6.38958586e-05, 6.36162503e-05])

In [135]:
prediction_Series.describe(include='all')

count    9.422000e+03
mean     6.349135e-05
std      3.730299e-08
min      6.274636e-05
25%      6.349198e-05
50%      6.349198e-05
75%      6.349198e-05
max      6.389586e-05
dtype: float64

In [174]:
prediction_Series_changed = prediction_Series


In [180]:
prediction_Series.describe

<bound method NDFrame.describe of 0       0.000063
1       0.000063
2       0.000063
3       0.000063
4       0.000063
          ...   
9417    0.000063
9418    0.000063
9419    0.000063
9420    0.000063
9421    0.000063
Length: 9422, dtype: float64>

In [181]:
type(prediction_Series_changed)

pandas.core.series.Series

In [157]:
#prediction_Series_changed = prediction_Series_changed.values.astype(np.int64)

In [182]:
prediction_Series_changed[prediction_Series_changed > 0] = 1

In [91]:
#prediction_Series_changed[prediction_Series_changed >= 0.0002381] = 0

In [183]:
pd.Series(prediction_Series_changed).describe(include='all')

count    9422.0
mean        1.0
std         0.0
min         1.0
25%         1.0
50%         1.0
75%         1.0
max         1.0
dtype: float64

In [119]:
test_['predictions'] = prediction_Series_changed[0:3178]

In [120]:
from math import sqrt
import sklearn.metrics as metrics
error = sqrt(metrics.mean_squared_error(X_test.values,predictions[0:3178]))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

### Обучим простой классификатор

In [194]:
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression #попробовали линейную регрессию

In [205]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import numpy as np
model = DecisionTreeRegressor(random_state=1)
model.fit(X_train.fillna(X_train.mean()),y_train)

DecisionTreeRegressor(random_state=1)

In [207]:
model.predict(X_train.head())

array([0., 0., 0., 0., 0.])

In [208]:
predicted_jam = model.predict(X_train.fillna(X_train.mean()))
mean_absolute_error(y_train, predicted_jam)

0.0

In [ ]:
X_train.head()

In [ ]:
#new_X_train = X_train.loc[:, ['year', 'station_id', 'day']]
new_X_train = X_train.loc[:, ['year', 'station_id', 'day']]

In [209]:
new_X_train['ice_jam'] = pd.DataFrame(predicted_jam).loc[:]

In [210]:
new_X_train.describe(include='all')

,year,station_id,day,ice_jam
count,6952.000000,6952.000000,6952.000000,6952.000000
mean,2001.196203,3045.221519,21.791139,0.010357
std,11.125786,46.292134,12.707463,0.101247
min,1986.000000,3019.000000,0.000000,0.000000
25%,1990.000000,3028.000000,11.000000,0.000000
50%,2000.000000,3030.000000,22.000000,0.000000
75%,2010.000000,3045.000000,33.000000,0.000000
max,2019.000000,3230.000000,44.000000,1.000000


In [211]:
new_X_train[['year', 'station_id', 'day', 'ice_jam']] = new_X_train[['year', 'station_id', 'day', 'ice_jam']].fillna(0.0).astype(int)

In [212]:
new_X_train.describe(include='all')

,year,station_id,day,ice_jam
count,6952.000000,6952.000000,6952.000000,6952.000000
mean,2001.196203,3045.221519,21.791139,0.010357
std,11.125786,46.292134,12.707463,0.101247
min,1986.000000,3019.000000,0.000000,0.000000
25%,1990.000000,3028.000000,11.000000,0.000000
50%,2000.000000,3030.000000,22.000000,0.000000
75%,2010.000000,3045.000000,33.000000,0.000000
max,2019.000000,3230.000000,44.000000,1.000000


In [213]:
new_X_train.reset_index(drop=True, inplace=True)

In [214]:
new_X_train.ice_jam.unique()

array([0, 1])

In [215]:
#df.to_csv('/home/codecraftman/notebook3.8/bin/out1.csv',index=False)
new_X_train.to_csv('/home/codecraftman/notebook3.8/bin/out4.csv',index=False)